In [15]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model


df = pd.read_csv('./data/pima-indians-diabetes3.csv')


df = pd.read_csv('./data/sonar3.csv',header=None)
df.head()
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]

#학습셋 테스트셋 분리
X_train , X_test, y_train,y_test = train_test_split(X,y,train_size=0.7,shuffle= True)


model = Sequential()
model.add(Dense(24, input_dim=X.shape[1], activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=200, verbose=False)

#모델 테스트셋 적용
score = model.evaluate(X_test,y_test)

print('Test accuracy : ',score[1])
print('Train accuracy : ',history.history['accuracy'][-1])
print(history.history['loss'][-1],history.history['accuracy'][-1])


2/2 [==============================] - 0s 7ms/step - loss: 0.4136 - accuracy: 0.7937
Test accuracy :  0.7936508059501648
Train accuracy :  0.9793103337287903
0.13542716205120087 0.9793103337287903


In [16]:
# 모델을 저장 재사용 방법
# 모델 load하는 함수
model.save('./data/my_model.hdf5')

del model
model = load_model('./data/my_model.hdf5')
score = model.evaluate(X_test,y_test)
print('Test accuracy',score[1])

2/2 [==============================] - 0s 12ms/step - loss: 0.4136 - accuracy: 0.7937
Test accuracy 0.7936508059501648


In [24]:
# 작은 데이터르 보완하기 위해 k-cross validation
from sklearn.model_selection import KFold

k = 5

kfold = KFold(n_splits=5,shuffle=True)


# for train_idx , test_idx in kfold.split(X):
#     print('train : ',train_idx)
#     print('test : ',test_idx)

def model_func(): #딥러닝 구현
    model = Sequential() # 딥러닝 모델 구현
    model.add(Dense(24,input_dim = 60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model
    

In [26]:
# k겹 교차검증을 이용한 k번의 학습 모델 실행

acc_score = []

for train_idx , test_idx in kfold.split(X):
    X_train = X.iloc[train_idx, :]
    X_test = X.iloc[test_idx, :]
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]

    
    model = model_func()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    hist = model.fit(X_train,y_train,epochs=200,batch_size=10,verbose=0)
    score = model.evaluate(X_test,y_test)[1] # 정확도 구현
    acc_score.append(score) # 정확도 리스트에 저장

avg_score = sum(acc_score)/k

# 결과 출력
print('정확도는 : ',acc_score)
print('정화도 평균 : ', avg_score)
    
    

2/2 [==============================] - 0s 7ms/step - loss: 0.4121 - accuracy: 0.8780
정확도는 :  [0.7142857313156128, 0.8809523582458496, 0.8809523582458496, 0.707317054271698, 0.8780487775802612]
정화도 평균 :  0.8123112559318543
